<a href="https://colab.research.google.com/github/tkazusa/keras_practice/blob/master/CAE_denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAEを使ったノイズ除去

In [0]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda

### MNISTデータセットの読み込みと前処理

In [5]:
from tensorflow.keras.datasets import mnist


(x_train, _), (x_test, _) = mnist.load_data() 
# CNNで扱いやすい形に変換
x_train = x_train.reshape(-1, 28, 28, 1) 
x_test = x_test.reshape(-1, 28, 28, 1) 
# 画像を0-1の範囲に正規化
x_train = x_train/255. 
x_test = x_test/255. 

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
array_to_img(x_train[0])

### マスキングノイズを加えて、擬似ノイズデータを生成

In [0]:
def make_masking_noise_data(data_x, percent=0.1):
    size = data_x.shape
    masking = np.random.binomial(n=1, p=percent, size=size)
    return data_x*masking

x_train_masked = make_masking_noise_data(x_train)
x_test_masked = make_masking_noise_data(x_test)

### ガウシアンノイズを加えて、擬似ノイズデータを生成

In [0]:
def make_gaussian_noise_data(data_x, scale=0.8):
    gaussian_data_x = data_x + np.random.normal(loc=0, scale=scale, size=data_x.shape)
    gaussian_data_x = np.clip(gaussian_data_x, 0, 1)
    return gaussian_data_x

x_train_gauss = make_gaussian_noise_data(x_train)
x_test_gauss = make_gaussian_noise_data(x_test)

### ノイズを加えた画像とオリジナル画像との比較

In [0]:
from IPython.display import display_png


display_png(array_to_img(x_train[0]))
display_png(array_to_img(x_train_gauss[0]))
display_png(array_to_img(x_train_masked[0]))

### CAEモデルの構築

In [15]:
def build_autoencoder():
  autoencoder = Sequential()

  # Encoder
  autoencoder.add(Conv2D(16, (3, 3), 1, activation='relu', padding='same', input_shape=(28, 28, 1)))
  autoencoder.add(MaxPooling2D((2, 2), padding='same'))
  autoencoder.add(Conv2D(8, (3, 3), 1, activation='relu', padding='same'))
  autoencoder.add(MaxPooling2D((2, 2), padding='same'))

  # Decoder
  autoencoder.add(Conv2D(8, (3, 3), 1, activation='relu', padding='same'))
  autoencoder.add(UpSampling2D((2, 2)))
  autoencoder.add(Conv2D(16, (3, 3), 1, activation='relu', padding='same'))
  autoencoder.add(UpSampling2D((2, 2)))
  autoencoder.add(Conv2D(1, (3, 3), 1, activation='sigmoid', padding='same'))

  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  
  return autoencoder

autoencoder = build_autoencoder()
initial_weights = autoencoder.get_weights()
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 16)        1168      
__________

### ガウシアンノイズを用いて学習

In [0]:
autoencoder.fit(
                x_train_gauss, # 入力：ガウシアンノイズデータ
                x_train,       # 正解：オリジナル画像
                epochs=10,     # 学習するエポック回数
                batch_size=20, # バッチサイズ
                shuffle=True   # シャッフルデータにする
               )

Epoch 1/10
60000/60000 [==============================] - 107s 2ms/step - loss: 3.5848
Epoch 2/10
60000/60000 [==============================] - 107s 2ms/step - loss: 3.0966
Epoch 3/10
60000/60000 [==============================] - 106s 2ms/step - loss: 3.0178
Epoch 4/10
60000/60000 [==============================] - 105s 2ms/step - loss: 2.9773
Epoch 5/10
60000/60000 [==============================] - 105s 2ms/step - loss: 2.9514
Epoch 6/10
56440/60000 [===========================>..] - ETA: 6s - loss: 2.9338

### ガウシアンノイズで学習したモデルでの予測

In [0]:
gauss_preds = autoencoder.predict(x_test_gauss)
for i in range(3):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_gauss[i]))
    display_png(array_to_img(gauss_preds[i]))
    print('-'*25)